# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 6 2023 12:33PM,258737,12,HH-40436,Hush Hush,Released
1,Mar 6 2023 12:33PM,258737,12,HH-40437,Hush Hush,Released
2,Mar 6 2023 12:33PM,258737,12,HH-40440,Hush Hush,Released
3,Mar 6 2023 12:33PM,258737,12,HH-40441,Hush Hush,Released
4,Mar 6 2023 12:33PM,258737,12,HH-40442,Hush Hush,Released
5,Mar 6 2023 12:33PM,258737,12,HH-40443,Hush Hush,Released
6,Mar 6 2023 12:33PM,258737,12,HH-40444,Hush Hush,Released
7,Mar 6 2023 12:29PM,258736,10,Enova-11465,Emerginnova,Released
8,Mar 6 2023 12:29PM,258735,19,9234460,"Snap Medical Industries, LLC",Released
9,Mar 6 2023 12:29PM,258735,19,9234461,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,258732,Released,11
26,258734,Released,1
27,258735,Released,3
28,258736,Released,1
29,258737,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258732,NaN,NaN,11.0
258734,NaN,NaN,1.0
258735,NaN,NaN,3.0
258736,NaN,NaN,1.0
258737,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258695,5.0,6.0,18.0
258697,0.0,1.0,1.0
258704,0.0,3.0,5.0
258705,0.0,1.0,0.0
258708,0.0,19.0,11.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258695,5,6,18
258697,0,1,1
258704,0,3,5
258705,0,1,0
258708,0,19,11


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258695,5,6,18
1,258697,0,1,1
2,258704,0,3,5
3,258705,0,1,0
4,258708,0,19,11


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258695,5,6,18
1,258697,,1,1
2,258704,,3,5
3,258705,,1,
4,258708,,19,11


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 6 2023 12:33PM,258737,12,Hush Hush
7,Mar 6 2023 12:29PM,258736,10,Emerginnova
8,Mar 6 2023 12:29PM,258735,19,"Snap Medical Industries, LLC"
11,Mar 6 2023 12:23PM,258734,19,"GUSA Granules USA, Inc."
12,Mar 6 2023 12:15PM,258732,10,"Snap Medical Industries, LLC"
23,Mar 6 2023 12:13PM,258731,12,"Digital Brands, LLC"
24,Mar 6 2023 12:13PM,258730,10,"Digital Brands, LLC"
38,Mar 6 2023 12:08PM,258729,10,Emerginnova
39,Mar 6 2023 12:06PM,258728,19,ACG North America LLC
40,Mar 6 2023 11:52AM,258727,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 6 2023 12:33PM,258737,12,Hush Hush,,,7
1,Mar 6 2023 12:29PM,258736,10,Emerginnova,,,1
2,Mar 6 2023 12:29PM,258735,19,"Snap Medical Industries, LLC",,,3
3,Mar 6 2023 12:23PM,258734,19,"GUSA Granules USA, Inc.",,,1
4,Mar 6 2023 12:15PM,258732,10,"Snap Medical Industries, LLC",,,11
5,Mar 6 2023 12:13PM,258731,12,"Digital Brands, LLC",,,1
6,Mar 6 2023 12:13PM,258730,10,"Digital Brands, LLC",,,14
7,Mar 6 2023 12:08PM,258729,10,Emerginnova,,,1
8,Mar 6 2023 12:06PM,258728,19,ACG North America LLC,,,1
9,Mar 6 2023 11:52AM,258727,12,Hush Hush,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 6 2023 12:33PM,258737,12,Hush Hush,7,,
1,Mar 6 2023 12:29PM,258736,10,Emerginnova,1,,
2,Mar 6 2023 12:29PM,258735,19,"Snap Medical Industries, LLC",3,,
3,Mar 6 2023 12:23PM,258734,19,"GUSA Granules USA, Inc.",1,,
4,Mar 6 2023 12:15PM,258732,10,"Snap Medical Industries, LLC",11,,
5,Mar 6 2023 12:13PM,258731,12,"Digital Brands, LLC",1,,
6,Mar 6 2023 12:13PM,258730,10,"Digital Brands, LLC",14,,
7,Mar 6 2023 12:08PM,258729,10,Emerginnova,1,,
8,Mar 6 2023 12:06PM,258728,19,ACG North America LLC,1,,
9,Mar 6 2023 11:52AM,258727,12,Hush Hush,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 6 2023 12:33PM,258737,12,Hush Hush,7,,
1,Mar 6 2023 12:29PM,258736,10,Emerginnova,1,,
2,Mar 6 2023 12:29PM,258735,19,"Snap Medical Industries, LLC",3,,
3,Mar 6 2023 12:23PM,258734,19,"GUSA Granules USA, Inc.",1,,
4,Mar 6 2023 12:15PM,258732,10,"Snap Medical Industries, LLC",11,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 6 2023 12:33PM,258737,12,Hush Hush,7.0,NaN,NaN
1,Mar 6 2023 12:29PM,258736,10,Emerginnova,1.0,NaN,NaN
2,Mar 6 2023 12:29PM,258735,19,"Snap Medical Industries, LLC",3.0,NaN,NaN
3,Mar 6 2023 12:23PM,258734,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
4,Mar 6 2023 12:15PM,258732,10,"Snap Medical Industries, LLC",11.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 6 2023 12:33PM,258737,12,Hush Hush,7.0,0.0,0.0
1,Mar 6 2023 12:29PM,258736,10,Emerginnova,1.0,0.0,0.0
2,Mar 6 2023 12:29PM,258735,19,"Snap Medical Industries, LLC",3.0,0.0,0.0
3,Mar 6 2023 12:23PM,258734,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
4,Mar 6 2023 12:15PM,258732,10,"Snap Medical Industries, LLC",11.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2328468,56.0,9.0,5.0
12,776195,9.0,0.0,0.0
15,258708,11.0,19.0,0.0
16,258705,0.0,1.0,0.0
19,1811057,30.0,7.0,0.0
20,258722,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2328468,56.0,9.0,5.0
1,12,776195,9.0,0.0,0.0
2,15,258708,11.0,19.0,0.0
3,16,258705,0.0,1.0,0.0
4,19,1811057,30.0,7.0,0.0
5,20,258722,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,56.0,9.0,5.0
1,12,9.0,0.0,0.0
2,15,11.0,19.0,0.0
3,16,0.0,1.0,0.0
4,19,30.0,7.0,0.0
5,20,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,56.0
1,12,Released,9.0
2,15,Released,11.0
3,16,Released,0.0
4,19,Released,30.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Completed,5.0,0.0,0.0,0.0,0.0,0.0
Executing,9.0,0.0,19.0,1.0,7.0,0.0
Released,56.0,9.0,11.0,0.0,30.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,19.0,1.0,7.0,0.0
2,Released,56.0,9.0,11.0,0.0,30.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,19.0,1.0,7.0,0.0
2,Released,56.0,9.0,11.0,0.0,30.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()